In [1]:
import numpy as np
from scipy.io import wavfile
import scipy.stats
import scipy

In [2]:
!git clone https://github.com/yassineboujrada/predict_gender_from_voice.git

Cloning into 'predict_gender_from_voice'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 16 (delta 1), reused 13 (delta 1), pack-reused 0
Unpacking objects: 100% (16/16), done.


In [5]:
!ls predict_gender_from_voice

app.py		  Daco_4884616.png  README.md	    tt.png     window.py
audio_data.ipynb  mm.py		    recording1.wav  voice.csv  yy.png


In [10]:
!cd ..

In [11]:
!ls

predict_gender_from_voice  sample_data


In [ ]:
def describe_freq(freqs):
    mean = np.mean(freqs)
    std = np.std(freqs) 
    maxv = np.amax(freqs) 
    minv = np.amin(freqs) 
    median = np.median(freqs)
    
    skew = scipy.stats.skew(freqs)
    kurt = scipy.stats.kurtosis(freqs)
    q1 = np.quantile(freqs, 0.25)
    q3 = np.quantile(freqs, 0.75)
    mode = scipy.stats.mode(freqs)[0][0]
    iqr = scipy.stats.iqr(freqs)
    
    return np.array( [[mean, std, median,q1, q3, minv,iqr, maxv]] ) #mean, std, maxv, minv, median,skew, kurt, q1, q3, mode, iqr]

output = wavfile.read('/predict_gender_from_voice/recording1.wav')
# frequence of audio it's output[0] but array it's output[1]
freqs = np.fft.fftfreq(output[1].size)
descr=describe_freq(output[1])
print(describe_freq(output[1]))
# print(output[0])

[[  2466.21793084   7790.92331475   2460.          -2402.
    7361.         -32768.           9763.          32767.        ]]


In [ ]:
# from sklearn import preprocessing
# scaler = preprocessing.MinMaxScaler().fit_transform(descr)
# scaler
import pandas as pd
df=pd.read_csv("/predict_gender_from_voice/voice.csv")
df.head(1)

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,0.0,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.0,0.0,male


In [ ]:
x_data=np.array(df[["meanfreq","sd","median","Q25","Q75","IQR","minfun","maxfun"]])
y_data=np.array(df["label"])
from sklearn.neighbors import KNeighborsClassifier 
import sklearn
x_train,x_test,y_train,y_test = sklearn.model_selection.train_test_split(x_data,y_data,test_size=0.1)
model=KNeighborsClassifier(n_neighbors=3)
model.fit(x_train,y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
df.describe()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
count,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000
mean,0.180907,0.057126,0.185621,0.140456,0.224765,0.084309,3.140168,36.568461,0.895127,0.408216,0.165282,0.180907,0.142807,0.036802,0.258842,0.829211,0.052647,5.047277,4.994630,0.173752
std,0.029918,0.016652,0.036360,0.048680,0.023639,0.042783,4.240529,134.928661,0.044980,0.177521,0.077203,0.029918,0.032304,0.019220,0.030077,0.525205,0.063299,3.521157,3.520039,0.119454
min,0.039363,0.018363,0.010975,0.000229,0.042946,0.014558,0.141735,2.068455,0.738651,0.036876,0.000000,0.039363,0.055565,0.009775,0.103093,0.007812,0.004883,0.007812,0.000000,0.000000
25%,0.163662,0.041954,0.169593,0.111087,0.208747,0.042560,1.649569,5.669547,0.861811,0.258041,0.118016,0.163662,0.116998,0.018223,0.253968,0.419828,0.007812,2.070312,2.044922,0.099766
50%,0.184838,0.059155,0.190032,0.140286,0.225684,0.094280,2.197101,8.318463,0.901767,0.396335,0.186599,0.184838,0.140519,0.046110,0.271186,0.765795,0.023438,4.992188,4.945312,0.139357
75%,0.199146,0.067020,0.210618,0.175939,0.243660,0.114175,2.931694,13.648905,0.928713,0.533676,0.221104,0.199146,0.169581,0.047904,0.277457,1.177166,0.070312,7.007812,6.992188,0.209183
max,0.251124,0.115273,0.261224,0.247347,0.273469,0.252225,34.725453,1309.612887,0.981997,0.842936,0.280000,0.251124,0.237636,0.204082,0.279114,2.957682,0.458984,21.867188,21.843750,0.932374


In [ ]:
max(df["meanfreq"].values)

0.251123758720282

In [18]:
def spectral_properties(y: np.ndarray, fs: int) -> dict:
    spec = np.abs(np.fft.rfft(y))
    freq = np.fft.rfftfreq(len(y), d=1 / fs)
    spec = np.abs(spec)
    amp = spec / spec.sum()
    mean = (freq * amp).sum()
    sd = np.sqrt(np.sum(amp * ((freq - mean) ** 2)))
    amp_cumsum = np.cumsum(amp)
    median = freq[len(amp_cumsum[amp_cumsum <= 0.5]) + 1]
    mode = freq[amp.argmax()]
    Q25 = freq[len(amp_cumsum[amp_cumsum <= 0.25]) + 1]
    Q75 = freq[len(amp_cumsum[amp_cumsum <= 0.75]) + 1]
    IQR = Q75 - Q25
    z = amp - amp.mean()
    w = amp.std()
    skew = ((z ** 3).sum() / (len(spec) - 1)) / w ** 3
    kurt = ((z ** 4).sum() / (len(spec) - 1)) / w ** 4

    result_d = {
        'mean': mean,
        'sd': sd,
        'median': median,
        'mode': mode,
        'Q25': Q25,
        'Q75': Q75,
        'IQR': IQR,
        'skew': skew,
        'kurt': kurt
    }

    return result_d

output = wavfile.read('predict_gender_from_voice/recording1.wav')
output[1].shape

(88200, 2)

In [29]:
def spectral_properties1(filename):

    fs, data = wavfile.read(filename)

    # use the first channel only
    if data.ndim > 1:
        data = data[:, 0]

    spec = np.abs(np.fft.rfft(data))
    freq = np.fft.rfftfreq(len(data), d=1/fs)

    assert len(spec) == len(freq)

    amp = spec / spec.sum()
    amp_cumsum = amp.cumsum()

    assert len(amp_cumsum) == len(freq)

    q25 = freq[len(amp_cumsum[amp_cumsum < 0.25])]
    q75 = freq[len(amp_cumsum[amp_cumsum < 0.75])]
    mean=(freq * amp).sum()
    median = freq[len(amp_cumsum[amp_cumsum <= 0.5]) + 1]
    sd = np.sqrt(np.sum(amp * ((freq - mean) ** 2)))
    IQR = q75 - q25
    z = amp - amp.mean()
    w = amp.std()
    skew = ((z ** 3).sum() / (len(spec) - 1)) / w ** 3
    kurt = ((z ** 4).sum() / (len(spec) - 1)) / w ** 4
    mode = freq[amp.argmax()]
    
    return mean, q25, q75,sd,median,IQR,skew,kurt,mode

spectral_properties1('predict_gender_from_voice/recording1.wav')

(1925.8766276770236,
 249.5,
 992.0,
 3981.0425737941914,
 481.0,
 742.5,
 26.294714597305266,
 1573.5817549303338,
 0.0)